In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import os
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def read_bin(path):
    #READING FROM FILE  
    dim=[0,0]
    fileData=[]
    with open(path, 'rb') as f:
        frame = int.from_bytes(f.read(4),byteorder = 'little')
        dim[0] = int.from_bytes(f.read(4),byteorder = 'little')
        dim[1] = int.from_bytes(f.read(4),byteorder = 'little')
        byteData = f.read(4)
        while byteData:  # データがあるだけ以下を繰り返す
            n = int.from_bytes(byteData, byteorder='little')  # 読み込んだデータを整数に変換
            fileData.append(n)  # リストに追加
            byteData = f.read(4)

    #CONVERTING TO DEPTH MAP FORMAT 
    depth = list(map(float,fileData))
    depthCountPerMap = dim[0] * dim[1]
    depthMap = np.zeros((frame,dim[1],dim[0]),dtype=float)
    for i in range(frame):
        startIndex = i * depthCountPerMap
        endIndex = (i+1) * depthCountPerMap
        currentDepthData = np.array(depth[startIndex:endIndex])
        depthMap[i,:,:] = currentDepthData.reshape(dim[1],dim[0])
    return depthMap

In [28]:
def process_one_action(path,ply=True):
    depth = read_bin(path)
    focal = 280
    xx, yy = np.meshgrid(np.arange(depth.shape[2]), np.arange(depth.shape[1]))

    point_clouds = []
    for d in range(depth.shape[0]):
        depth_map = depth[d]

        depth_min = depth_map[depth_map > 0].min()
        depth_map = depth_map

        x = xx[depth_map > 0]
        y = yy[depth_map > 0]
        z = depth_map[depth_map > 0]
        x = (x - depth_map.shape[1] / 2) / focal * z
        y = (y - depth_map.shape[0] / 2) / focal * z

        points = np.stack([x, y, z], axis=-1)

        point_clouds.append(points)
    if ply:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(np.array(point_clouds[0]))
        o3d.io.write_point_cloud("DATA/" + path.split('/')[2][:-4] + ".ply", pcd)
    else:
        np.savez_compressed("DATA/" + path.split('/')[2][:-4] + '.npz', point_clouds=point_clouds[0])

In [25]:
def show_pointclouds(point_clouds):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.array(point_clouds[0]))
    o3d.visualization.draw_geometries([pcd])

In [26]:
path = 'MSRAction3D/Depth/a01_s01_e01_sdepth.bin'
process_one_action(path,ply=False)

In [32]:
import multiprocessing

input_dir = 'MSRAction3D/Depth/'
files = os.listdir(input_dir)
pool = multiprocessing.Pool(2)

for input_file in files:
    #print(input_file)
    # process_one_file(os.path.join(input_dir, input_file))
    pool.apply_async(process_one_action,(os.path.join(input_dir, input_file),True))

pool.close()
pool.join()

a08_s03_e03_sdepth.bin
a08_s09_e03_sdepth.bin
a13_s05_e03_sdepth.bin
a10_s10_e01_sdepth.bin
a13_s09_e01_sdepth.bin
a15_s09_e01_sdepth.bin
a12_s04_e02_sdepth.bin
a18_s10_e03_sdepth.bin
a04_s01_e02_sdepth.bin
a18_s01_e03_sdepth.bin
a08_s01_e03_sdepth.bin
a02_s10_e02_sdepth.bin
a01_s05_e01_sdepth.bin
a08_s06_e01_sdepth.bin
a12_s01_e02_sdepth.bin
a09_s01_e02_sdepth.bin
a09_s07_e02_sdepth.bin
a12_s02_e02_sdepth.bin
a13_s04_e03_sdepth.bin
a10_s07_e03_sdepth.bin
a18_s08_e03_sdepth.bin
a19_s06_e02_sdepth.bin
a13_s01_e03_sdepth.bin
a20_s04_e01_sdepth.bin
a06_s01_e01_sdepth.bin
a16_s08_e01_sdepth.bin
a16_s10_e01_sdepth.bin
a18_s05_e03_sdepth.bin
a10_s03_e03_sdepth.bin
a19_s04_e02_sdepth.bin
a13_s06_e03_sdepth.bin
a06_s03_e01_sdepth.bin
a15_s02_e01_sdepth.bin
a02_s05_e02_sdepth.bin
a14_s10_e02_sdepth.bin
a08_s06_e03_sdepth.bin
a18_s09_e02_sdepth.bin
a17_s07_e01_sdepth.bin
a10_s08_e01_sdepth.bin
a20_s08_e01_sdepth.bin
a06_s09_e01_sdepth.bin
a20_s02_e02_sdepth.bin
a01_s02_e02_sdepth.bin
a11_s03_e02